# Dataloader experiments

In [5]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt

from pointnet2_ops import pointnet2_utils
from knn_cuda import KNN
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## 3D PCD: Importing, Downsampling and Saving as numpy arrays

In [4]:
print("Loading Open3D point cloud")
path = "../dataset/OldDataset/Barn_is/Barn/Barn02.ply"
pcd = o3d.io.read_point_cloud(path)
print("PCD Details:")
print(pcd)

# Link: https://www.open3d.org/docs/latest/tutorial/Advanced/pointcloud_outlier_removal.html
print("Every 100th points are selected, could use voxel or uniform downsampling")
down_pcd = pcd.voxel_down_sample(voxel_size=0.001)
# down_pcd = pcd.uniform_down_sample(every_k_points=10)

#Link: https://www.open3d.org/html/tutorial/Basic/working_with_numpy.html
np_pcd = np.asarray(down_pcd.points)
print(np_pcd.shape)


print(os.path.exists(path[:-10]))
np.save(path[:-3] + "npy", np_pcd)

#Visualise PCD
o3d.visualization.draw_geometries([down_pcd])


Loading Open3D point cloud
PCD Details:
PointCloud with 3305027 points.
Every 100th points are selected, could use voxel or uniform downsampling
(314410, 3)
True


In [4]:
# Using fps to make Nx3 to npointx3(usually 8192x3) 
def farthest_point_sample(point, npoint):
    """
    Input:
        xyz: pointcloud data, [N, D]
        npoint: number of samples
    Return:
        centroids: sampled pointcloud index, [npoint, D]
    """
    N, D = point.shape
    xyz = point[:,:3]
    centroids = np.zeros((npoint,))
    distance = np.ones((N,)) * 1e10
    farthest = np.random.randint(0, N)
    for i in range(npoint):
        centroids[i] = farthest
        centroid = xyz[farthest, :]
        dist = np.sum((xyz - centroid) ** 2, -1)
        mask = dist < distance
        distance[mask] = dist[mask]
        farthest = np.argmax(distance, -1)
    point = point[centroids.astype(np.int32)]
    return point

### Prepping 8192x3 float tensors

In [5]:
path = "../dataset/Dataset/pcd/Barn02.npy"
np_pcd = np.load(path)

print(np_pcd.shape)

fps_pcd = farthest_point_sample(np_pcd , 8192)
print(fps_pcd.shape)


tensor_pcd = torch.from_numpy(np.reshape(fps_pcd, (1, fps_pcd.shape[0], fps_pcd.shape[1]))).to(device).cuda().float()

print("Loaded point cloud of shape", tensor_pcd.dtype)

(16384, 3)
(8192, 3)
Loaded point cloud of shape torch.float32


In [6]:
class Tokenizer (nn.Module):
    def __init__(self, num_group, group_size):
        super().__init__()
        self.num_group = num_group
        self.group_size = group_size
        self.knn = KNN(k=self.group_size, transpose_mode=True)

    def forward(self, xyz):
            '''
                input: B N 3
                ---------------------------
                output: B G M 3
                center : B G 3
            '''
            batch_size, num_points, _ = xyz.shape
            # fps the centers out
            center = self.fps(xyz, self.num_group) # B G 3
            # knn to get the neighborhood
            _, idx = self.knn(xyz, center) # B G M
            assert idx.size(1) == self.num_group
            assert idx.size(2) == self.group_size
            idx_base = torch.arange(0, batch_size, device=xyz.device).view(-1, 1, 1) * num_points
            idx = idx + idx_base
            idx = idx.view(-1)
            neighborhood = xyz.view(batch_size * num_points, -1)[idx, :]
            neighborhood = neighborhood.view(batch_size, self.num_group, self.group_size, 3).contiguous()
            # normalize
            neighborhood = neighborhood - center.unsqueeze(2)
            return neighborhood, center
    
    def fps(self, data, number):
        '''
            data B N 3
            number int
        '''
        # print(number)
        # print("yoyoyo",data.scalar_type())
        fps_idx = pointnet2_utils.furthest_point_sample(data, number) 
        fps_data = pointnet2_utils.gather_operation(data.transpose(1, 2).contiguous(), fps_idx).transpose(1,2).contiguous()
        print(fps_data)
        return fps_data


### Transforms for PointClouds

In [7]:
from torchvision import transforms

class PointcloudScaleAndTranslate(object):
    def __init__(self, scale_low=2. / 3., scale_high=3. / 2., translate_range=0.2):
        self.scale_low = scale_low
        self.scale_high = scale_high
        self.translate_range = translate_range

    def __call__(self, pc):
        bsize = pc.size()[0]
        for i in range(bsize):
            xyz1 = np.random.uniform(low=self.scale_low, high=self.scale_high, size=[3])
            xyz2 = np.random.uniform(low=-self.translate_range, high=self.translate_range, size=[3])
            
            pc[i, :, 0:3] = torch.mul(pc[i, :, 0:3], torch.from_numpy(xyz1).float().cuda()) + torch.from_numpy(xyz2).float().cuda()
            
        return pc


train_transforms = transforms.Compose(
    [
        # data_transforms.PointcloudScale(),
        # data_transforms.PointcloudRotate(),
        # data_transforms.PointcloudRotatePerturbation(),
        # data_transforms.PointcloudTranslate(),
        # data_transforms.PointcloudJitter(),
        # data_transforms.PointcloudRandomInputDropout(),
        PointcloudScaleAndTranslate(),
    ]
)



### Tokenizer forward pass

In [8]:
tokenizer = Tokenizer(32,64)

In [9]:
tensor_pcd = train_transforms(tensor_pcd)

In [10]:
test = tokenizer.forward(tensor_pcd)

print(test[0].shape)
print(test[1].shape)

tensor([[[ -4.3732,  -4.0375, -34.9515],
         [-20.5198,  -4.7560, -35.4129],
         [ -3.5903,   4.8602, -28.0963],
         [-12.5905,  -1.1428, -29.4915],
         [  3.3627,  -5.5521, -31.8151],
         [ -4.4329,  -4.1252, -27.2561],
         [-11.8513,  -5.5845, -35.6064],
         [-19.1418,  -1.2476, -29.6718],
         [-15.7147,  -4.7538, -31.9732],
         [ -8.1887,  -3.9590, -31.0826],
         [  0.1067,  -6.0192, -35.5275],
         [ -1.3346,  -4.1177, -31.0832],
         [ -2.7092,   0.3938, -28.8218],
         [ -7.8919,  -5.6614, -35.5239],
         [-11.9187,  -3.8655, -32.1039],
         [-16.8353,  -5.4843, -35.5011],
         [  3.9335,  -5.5425, -35.5261],
         [ -4.7668,  -4.8317, -31.4273],
         [-19.0940,  -4.2970, -31.5667],
         [-15.8823,  -0.9626, -29.4315],
         [ -9.6888,  -3.9202, -34.0051],
         [ -1.6371,  -4.1008, -34.0073],
         [  0.6305,  -5.4310, -32.7284],
         [ -6.6380,  -4.9478, -28.9244],
         [ -6.80

### Optional visualize numpy array pcds 

In [11]:
import open3d as o3d


# Pass xyz to Open3D.o3d.geometry.PointCloud and visualize
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(np_pcd)
o3d.visualization.draw_geometries([pcd])

# o3d.io.write_point_cloud("../../TestData/sync.ply", pcd)
